In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
import seaborn as sns
import albumentations as A
from skimage.exposure import rescale_intensity


import numpy as np
import matplotlib.pyplot as plt

from skimage import data
from skimage.color import rgb2hed, hed2rgb

In [3]:
def imageProcessing1(df):

        #img = cv2.imread(df)
        # Example IHC image
        ihc_rgb = df#img

        # Separate the stains from the IHC image
        ihc_hed = rgb2hed(ihc_rgb)

        # Create an RGB image for each of the stains
        null = np.zeros_like(ihc_hed[:, :, 0])
        ihc_h = hed2rgb(np.stack((ihc_hed[:, :, 0], null, null), axis=-1))

        ihc_h = ihc_h*255      

   
        image1 = ihc_h.astype('uint8')
   
        # cv2.cvtColor is applied over the
        # image input with applied parameters
        # to convert the image in grayscale 
        image_result = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)


        return image_result

Declaration of a simple preprocessor function.  Preprocessor functions are used to preprocess an instance before labeling it. We use @preprocessor(name,resources) decorator to declare a function as preprocessor.

In [4]:
#from spear.labeling import preprocessor

#@preprocessor(name = "noise_removal")
def imagePreProcessing(df):

        img = cv2.imread(df)
        #Noise Removing
        image = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)
        #Gaussian Blur
        gaussian_3 = cv2.GaussianBlur(image, (9,9), 10.0) #unblur
        image = cv2.addWeighted(image, 1.5, gaussian_3, -0.5, 0, image)
        #Laplacian Filter
        #kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
        #image = cv2.filter2D(image, -1, kernel)
        return image

In [5]:
#@labeling_function(resources=dict(keywords=trigWord1),pre=[convert_to_lower],label=ClassLabels.SPAM)
def LF1(c): 
    
    #image = d
    img = c
    # applying different thresholding 
    # techniques on the input image
    # Otsu's thresholding after Gaussian filtering
    # Apply GaussianBlur to reduce image noise if it is required

    blur = cv2.GaussianBlur(img,(5,5),0)
    otsu_threshold, image_result = cv2.threshold(
         blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU,)
    
    thresh2 = 255-image_result

    output = cv2.connectedComponentsWithStats(thresh2)
    (numLabels, labels, stats, centroids) = output
    mask = np.zeros(thresh2.shape, dtype="uint8")

    # loop over the number of unique connected component labels, skipping
    # over the first label (as label zero is the background)
    for i in range(1, numLabels):
        # extract the connected component statistics for the current label
        x = stats[i, cv2.CC_STAT_LEFT]
        y = stats[i, cv2.CC_STAT_TOP]
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]
        area = stats[i, cv2.CC_STAT_AREA]
    
        # ensure the width, height, and area are all neither too small
        # nor too big
        #keepWidth = w > 5 and w < 30
        #keepHeight = h > 5 and h < 30
        #keepArea = area > 10

        keepWidth = w > 20
        keepHeight = h > 20
        keepArea = 100 < area < 2500
        # ensure the connected component we are examining passes all
        # three tests
        if all((keepWidth, keepHeight, keepArea)):
            # construct a mask for the current connected component and
            # then take the bitwise OR with the mask       
            componentMask = (labels == i).astype("uint8") * 1
            mask = cv2.bitwise_or(mask, componentMask)


        # Multiple objects
        #result = image.copy()
        contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = contours[0] if len(contours) == 2 else contours[1]
        colour = (255, 0, 0)
        thickness = 1
        i = 0
        
        bounding_boxes = []
       # Iterate through the contours and find bounding boxes
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            bounding_boxes.append([x, y, x+w, y+h])
            #cv2.rectangle(result, (x, y), (x+w, y+h), colour, thickness)
  

    return bounding_boxes


In [ ]:
import random
from PIL import Image

# Select 100 random images from the range of image numbers
random_indices = random.sample(range(500), 500)

# Specify the folder path where the images should be saved
folder_path = '/content/drive/MyDrive/CAGE-Unsupervised'

# Iterate through the selected images
for index in random_indices:
    # Load the image using the index
    image_path = 'image_{}.png'.format(index)
    im = Image.open(image_path)
    
    #cv2.imwrite('temp.jpg',np.array(im))
    #pre_processed = imagePreProcessing('temp.jpg')

    pre_processed = imagePreProcessing(image_path)
    processed1 = imageProcessing1(pre_processed)
    bounding_boxes1 = LF1(processed1)

    # Get the bounding boxes for the image
    #bounding_boxes1 = LF1(im)
    # Iterate through the bounding boxes
    for i, box in enumerate(bounding_boxes1):
        x1, y1, x2, y2 = box
        # Crop the image to the bounding box
        cropped_im = im.crop((x1, y1, x2, y2))
        # Save the cropped image with a unique name
        cropped_im.save(f"{folder_path}/image-{index}_box_{i}.png")